In [75]:
import tensorflow as tf
import numpy as np
import os
import cv2
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tf.compat.v1.disable_eager_execution()

Funcionalidade das bibliotecas que foram utilizadas:

- Numpy: Facilita algumas operações e manipulações matematicas, e permite a captura da forma vetorial das imagens.
- os: Fornece algumas funções de manipulação do proprio sistema operacional que foram uteis para testes feitos antes da integração com a base de dados.
- cv2: Permite o redimensionamento das imagens.
- sklearn: Biblioteca de aprendizagem de maquina da qual foi utilizada as funções que implementaram o perceptron
- TensorFlow





Considerando uma implementação de teste sem integração com a base de dados:

É pego a lista de diretórios do caminho atual, e então atribuido a uma lista. Esses diretórios são as categorias de animais que serão avaliadas, no caso, gatos e cachorros.

In [76]:
Animals_types = os.listdir('Animals//')
Animals_types

['cats', 'dogs']

Então, para cada diretório que foi descoberto é feita uma varredura, e atribuida cada imagem que está no formato jpg para a lista de imagens, e a sua categoria correspondente para a lista de labels.

In [77]:
path = 'Animals//'

sizeExpected = 60

images = []
labels = []

for i in Animals_types:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path) if i.endswith('.jpg')]
    print(filenames)
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img,(sizeExpected, sizeExpected))
        images.append(img)
        labels.append(i)

['CAT1.jpg', 'CAT2.jpg']
['DOG 1.jpg', 'DOG 2.jpg']


Com isso, pode-se converter as imagens em questão para sua forma vetorial, garantir que estejam no formato correto (Float32) e sobre a escala de cores RGB (255)

Com a função shape podemos ver que o primeiro elemento da 4-tupla é a quantidade de imagens que foram pegas, os proximos dois são a dimensão que foi escolhida para análise, e o ultimo elemento indica que se trata de uma imagem colorida (RGB).

In [78]:
images = np.array(images)
images = images.astype('float32')/255
images.shape

(4, 60, 60, 3)

Com as funções fornecidas pela biblioteca sklearn podemos começar a criação e atribuição de labels para o treinamento. Dessa forma, a variavel x recebe a lista de imagens pegas, e a y recebe a lista binária de labels (o que permite que as labels sejam "entendidas" pelo computador), onde no nosso caso, seria:
- 0 para Gatos
- 1 para Cachorros.

In [79]:
x = images
y = labels
y_labelEncoder = LabelEncoder()
y = y_labelEncoder.fit_transform(y)
print(y)

[0 0 1 1]


Nesse momento é separado as labels em duas classes já que precisamos de duas colunas de classificação.

In [80]:
y = y.reshape(-1,1)

ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
y = ct.fit_transform(y)
print(y)

[[1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]


Neste momento é feita a separação entre treino e teste.

In [81]:
x,Y = shuffle(x,y, random_state=1)
train_x, test_x, train_y, test_y = train_test_split(images, y, test_size=0.20, random_state=415)

In [82]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(3, 60, 60, 3)
(3, 2)
(1, 60, 60, 3)
(1, 2)


Como o objetivo é utilizar um perceptron para resolver o problema da classificação, precisamos transformar o formato da imagem para apenas uma dimensão. Dessa forma, temos que:

Nova dimensão = (n, r * r * 3)

Sendo o n o valor do primeiro elemento da 4-tupla da célula anterior (o numero de imagens do treino e do teste), e r o numéro que redimensionamos a imagem anteriormente.

Logo:
- Para o treino:    Nova dimensão = (3, 10800)
- Para o teste:     Nova dimensão = (1, 10800)

In [83]:
train_x=np.reshape(train_x,(3,10800))
print(train_x.shape)

test_x=np.reshape(test_x,(1,10800))
test_x.shape

(3, 10800)


(1, 10800)

In [84]:
learning_rate = 0.1
training_epochs = 100
n_dim=60
n_class=2


Perceptron_x = tf.compat.v1.placeholder(tf.float32, [None, 10800])
Perceptron_W = tf.Variable(tf.zeros([10800,n_class]))
Perceptron_b = tf.Variable(tf.zeros([n_class]))
Perceptron_y = tf.compat.v1.placeholder(tf.float32,[None,n_class]) 

In [85]:
init = tf.compat.v1.global_variables_initializer()
pred = tf.nn.softmax(tf.matmul(Perceptron_x, Perceptron_W)+ Perceptron_b) # perceptron
 
error_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Perceptron_y)) 
training_step = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(error_function)


In [86]:

sess = tf.compat.v1.Session()
sess.run(init)

cost_history = []

In [87]:
for epoch in range(training_epochs):
    sess.run(training_step,feed_dict={Perceptron_x:train_x,Perceptron_y:train_y})
    cost = sess.run(error_function,feed_dict={Perceptron_x: train_x,Perceptron_y: train_y})
    cost_history = np.append(cost_history,cost)
    print('epoch : ', epoch,  ' - ', 'cost: ', cost)


epoch :  0  -  cost:  0.31327066
epoch :  1  -  cost:  0.31327063
epoch :  2  -  cost:  0.3132706
epoch :  3  -  cost:  0.3132706
epoch :  4  -  cost:  0.31327057
epoch :  5  -  cost:  0.3132705
epoch :  6  -  cost:  0.3132705
epoch :  7  -  cost:  0.31327045
epoch :  8  -  cost:  0.31327045
epoch :  9  -  cost:  0.31327042
epoch :  10  -  cost:  0.3132704
epoch :  11  -  cost:  0.31327036
epoch :  12  -  cost:  0.31327036
epoch :  13  -  cost:  0.31327033
epoch :  14  -  cost:  0.3132703
epoch :  15  -  cost:  0.3132703
epoch :  16  -  cost:  0.31327024
epoch :  17  -  cost:  0.31327024
epoch :  18  -  cost:  0.31327024
epoch :  19  -  cost:  0.3132702
epoch :  20  -  cost:  0.31327018
epoch :  21  -  cost:  0.31327018
epoch :  22  -  cost:  0.31327012
epoch :  23  -  cost:  0.31327012
epoch :  24  -  cost:  0.3132701
epoch :  25  -  cost:  0.31327006
epoch :  26  -  cost:  0.31327003
epoch :  27  -  cost:  0.31327
epoch :  28  -  cost:  0.31327
epoch :  29  -  cost:  0.31326997
epoch

In [88]:
test_x

array([[1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [91]:

image = load_img('algo2.jpg', target_size=(60,60))
image = np.array(image)

image.shape
image = np.reshape(image,(1,10800))

In [92]:
result = sess.run(pred, feed_dict={Perceptron_x: image})
print(result)

if(result[0][0] > result[0][1]):
    print("Gato")
else:
    print("Cachorro")

[[0. 1.]]
Cachorro
